In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob

"""Change to the data folder"""
new_path = "./new_train/new_train"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    return [inp, out]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

### Visualize the batch of sequences

In [4]:
import matplotlib.pyplot as plt
import random

agent_id = 0

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

        
for i_batch, sample_batch in enumerate(val_loader):
    inp, out = sample_batch
    show_sample_batch(sample_batch, agent_id)
    break

## Build CNN model

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(60, 30, 3, stride=1)
        #self.conv2 = nn.Conv2d(30, 10, 3, stride=1)
        
        self.fc1 = nn.Linear(15*16*2, 100)
        self.fc2 = nn.Linear(100, 84)
        self.fc3 = nn.Linear(84, 10)
        
        self.pool = nn.AvgPool2d(2, stride=1) 

    def forward(self, x):
#         print("x begin = ", x.shape)
#         x = F.relu(self.conv1(x))
#         print("x after conv1 =", x.shape)
#         x = torch.reshape(x, (4,30, 34))
#         print("x after reshape =", x.shape)
        
#         x = self.conv1(x)
#         x = self.pool(x)
#         x = F.relu(x)
        
#         #x = F.relu((self.pool(self.conv2(x))))

#         x = x.view(-1, 15*16*2)
        
#         x = self.fc1(x)
#         x = self.fc2(x)
#         x = self.fc3(x)
        
        return F.log_softmax(x,dim=1)

## Build RNN Model

In [21]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your RNN
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='tanh')

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)

        # One time step
        # We need to detach the hidden state to prevent exploding/vanishing gradients
        # This is part of truncated backpropagation through time (BPTT)
        out, hn = self.rnn(x, h0.detach())

        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out

In [7]:
input_dim = 19
hidden_dim = 100
layer_dim = 2  # ONLY CHANGE IS HERE FROM ONE LAYER TO TWO LAYER
output_dim = 10

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

#######################
#  USE GPU FOR MODEL  #
#######################

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)


'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()

'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.03

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  


In [22]:
# Number of steps to unroll
seq_dim = 19
num_epochs = 1
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(val_loader):
        # Load images as tensors with gradient accumulation abilities
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        images = images.view(4, 60, input_dim, 4).requires_grad_().to(device)
        labels = labels.to(device)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

#         if iter % 500 == 0:
#             # Calculate Accuracy         
#             correct = 0
#             total = 0
#             # Iterate through test dataset
#             for images, labels in test_loader:
#                 #######################
#                 #  USE GPU FOR MODEL  #
#                 #######################
#                 images = images.view(-1, seq_dim, input_dim).to(device)

#                 # Forward pass only to get logits/output
#                 outputs = model(images)

#                 # Get predictions from the maximum value
#                 _, predicted = torch.max(outputs.data, 1)

#                 # Total number of labels
#                 total += labels.size(0)

#                 # Total correct predictions
#                 #######################
#                 #  USE GPU FOR MODEL  #
#                 #######################
#                 if torch.cuda.is_available():
#                     correct += (predicted.cpu() == labels.cpu()).sum()
#                 else:
#                     correct += (predicted == labels).sum()

#             accuracy = 100 * correct / total

            # Print Loss
        print('Iteration: {}. Loss: {}.' .format(iter, loss.item()))

RuntimeError: input must have 3 dimensions, got 4

 ## Define train function

In [6]:
from tqdm import tqdm_notebook as tqdm
def train(model, device, val_loader, optimizer, epoch, log_interval=10000):
    model.train()
    iterator = tqdm(val_loader, total=int(len(val_loader)))
    counter = 0
    
    for batch_idx, (data, target) in enumerate(iterator):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        print("output = ", output.shape)
        print("target = ", target.shape)
        
        target = torch.flatten(target,start_dim = 2)
        
        loss = F.nll_loss(output, target)
        
        loss.backward()
        optimizer.step()
        counter += 1
        iterator.set_postfix(loss=(loss.item()*data.size(0) / (counter * val_loader.batch_size)))

## Training deep learning model

In [7]:
learning_rate = 0.01
momentum = 0.5
device = "cpu"
model = CNN().to(device) #using cpu here
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)
num_epoch = 10

print("Number of param: ", sum(p.numel()for p in model.parameters()))

for epoch in range(1, num_epoch + 1):
        train(model, device, val_loader, optimizer, epoch)

Number of param:  73664


output =  torch.Size([4, 60, 19, 4])
target =  torch.Size([4, 60, 30, 4])


RuntimeError: size mismatch (got input: [4, 60, 19, 4] , target: [4, 60, 120]